<a href="https://colab.research.google.com/github/AtacZy/Projects/blob/main/%D0%9E%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0_%D0%BF%D1%80%D0%BE%D1%81%D1%80%D0%BE%D1%87%D0%B5%D0%BA_%D0%BF%D0%BB%D0%B0%D1%82%D0%B5%D0%B6%D0%B5%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Обработка просрочек платежей

##**Цель**:
Собрать данные с Google Sheets и написать функцию generate_report, которая принимает три списка (данные с каждого листа в Google Sheets) и сохраняет в файл student_debt_report.txt

##**Данные**:
**Лист1**: student_id - айди студента; student_name - имя студента; installment (Y/N) - факт наличия рассрочки у студента (Y - рассрочка есть, N - рассрочки нет).

**Лист2**: student_id - айди студента; last_payment_date - дата последней полученной оплаты; expected_payment_date - дата, в которую ожидаем следующий платеж (рассчитывается от даты последнего платежа).

**Лист3**: student_id - айди студента; already_payed_amount - уже выплаченная часть рассрочки; left_to_pay - оставшаяся (невыплаченная) часть; one-time_payment - единоразовый платеж; installment_amount - общая сумма, которая бралась в рассрочку.

##**Примечание**:
Считать должников необходимо на 1 марта 2023 года. То есть определяя количество просроченных платежей, мы определяем их количество не на настоящую дату, а на 1 марта 2023 года. А периоды внесения платежей для всех студентов одинаковы - 6 месяцев (183 дня).

##**Логика работы**
1. Импортировать данные с Google Sheets и поместить в списки;
2. Собрать данные с трёх списков в один словарь, который будет представлен в виде определённого шаблона;
3. Необходимо проверить просрочки платежей в получившемся списке. Учитываем несколько просрочек;
4. Записать получившийся список или словарь в отчёт
5. Запустить функцию

##**Выводы**:
1. Поработали с импортом данных с Google Sheets;
2. Преобразовали импортированные данные в код для написания функции generate_report
3. Получили файл txt, где хранится информация о просрочках платежей




##**Как должны выгялдеть данные в 'student_debt_report'**: <br>
Студент Иванов У.У. - долг 100000 рублей <br>
Студент Петров Е.Е. - долг 250000 рублей <br>
Студент Сидоров И.И. - долг 10000 рублей

**Импортируем данные**

In [ ]:
!wget https://gist.github.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json

!wget https://gist.github.com/Vs8th/39c5deed0f5539d781f00328f7fd4fe0/raw/result.txt

 **Считываем с помощью библиотеки gspread данные и помещаем каждый лист в отдельный список**

In [49]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://www.googleapis.com/auth/spreadsheets.readonly',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('creds.json', scope)
client = gspread.authorize(creds)
spreadsheet = client.open_by_key('1hRnw-PEftF0J-6KY7InlILVwWdkJu4vJiGwRjuE_P4U')
sheet1_data = spreadsheet.worksheet('Лист1').get_all_records()
sheet2_data = spreadsheet.worksheet('Лист2').get_all_records()
sheet3_data = spreadsheet.worksheet('Лист3').get_all_records()

**Пишем функцию, где читаем каждый список и помещаем по критериям в один единый словарь или список**

In [50]:
from datetime import datetime
from datetime import timedelta
import math

def generate_report(sheet1, sheet2, sheet3):
    report = {}

# Собираем данные с трех листов и сохраняем в словарь
    for student in sheet1:
        student_id = student['student_id']
        name = student['student_name']
        installment = student['installment']

        report[student_id] = {'name': name, 'debt': 0, 'last_payment_date': None}

    for payment in sheet2:
        student_id = payment['student_id']
        last_payment_date = datetime.strptime(payment['last_payment_date'], '%d.%m.%Y')
        expected_payment_date = datetime.strptime(payment['expected_payment_date'], '%d.%m.%Y')
        if student_id in report:
            report[student_id]['last_payment_date'] = last_payment_date
            report[student_id]['expected_payment_date'] = expected_payment_date

    for debt in sheet3:
        student_id = debt['student_id']
        already_payed_amount = debt['already_payed_amount']
        left_to_pay = debt['left_to_pay']
        one_time_payment = debt['one-time_payment']
        installment_amount = debt['installment_amount']

# Проверяем просрочен ли ожидаемый платеж. Учитываем несколько просрочек.
        if student_id in report and report[student_id]['expected_payment_date'] < datetime(2023, 3, 1):
            period = datetime(2023, 3, 1) - report[student_id]['expected_payment_date']
            debt = one_time_payment * (math.ceil(period/timedelta(days=183)))
            if debt <= left_to_pay:
                report[student_id]['debt'] = debt
            else:
                report[student_id]['debt'] = left_to_pay

# Записываем отчет в файл
    with open("student_debt_report.txt", "w") as file:
        for student_id, data in report.items():
            if data['debt'] == 0:
                continue
            else:
                name = data['name']
                debt = data['debt']
                file.write(f"Студент {name} - долг {debt} рублей\n")

generate_report(sheet1_data, sheet2_data, sheet3_data)

**Получившийся файл:**

In [ ]:
with open('student_debt_report.txt', 'r') as file:
  print(file.read())

**Запустить, дял проверки:**

In [ ]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt

import pandas as pd

user_answer = pd.read_csv('student_debt_report.txt')
correct_answer = pd.read_csv('student_debt.txt')

In [ ]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
  assert user_answer.columns.equals(correct_answer.columns), 'Названия столбцов не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')